CASE DIGITAL BEMOL

AS PROJEÇÕES FORAM PONDERADAS COM A SAZONALIDADE DAS DATAS COMEMORATIVAS BASEADA EM INTELIGÊNCIA ARTIFICIAL, O IDEIAL É QUE TENHAMOS UM BASELINE DE 12 MESES NO MÍNIMO PARA AVALIAÇÃO DE TENDÊNCIAS REAIS

# STEP 01 - IDENTIFICAÇÃO DO PROBLEMA

Qual é o problema, sua abrangência e seu impacto. Use perguntas como "Por que isso é um problema?" e "Quais são as restrições?"

In [ ]:
# Dashboard no próprio Colab com Plotly
import pandas as pd
import plotly.express as px
from datetime import timedelta
import numpy as np
import matplotlib.pyplot as plt

base_path = "/content/drive/MyDrive/home/ubuntu/"

# Carregamento dos dados
pedidos = pd.read_csv(base_path + "pedidos.csv", parse_dates=["data_hora_criacao"])
itens = pd.read_csv(base_path + "itens_pedido.csv")
separacao = pd.read_csv(base_path + "log_separacao.csv", parse_dates=["ts_inicio_separacao", "ts_fim_separacao"])
expedicao = pd.read_csv(base_path + "log_expedicao.csv", parse_dates=["ts_pronto_expedicao", "ts_inicio_carregamento", "ts_fim_expedicao"])

# Limpeza de dados
pedidos.dropna(inplace=True)
itens.dropna(inplace=True)
separacao.dropna(subset=["ts_inicio_separacao", "ts_fim_separacao", "status_separacao"], inplace=True)
expedicao.dropna(subset=["ts_pronto_expedicao", "ts_inicio_carregamento"], inplace=True)

separacao = separacao[separacao["ts_fim_separacao"] >= separacao["ts_inicio_separacao"]]
expedicao = expedicao[expedicao["ts_inicio_carregamento"] >= expedicao["ts_pronto_expedicao"]]

# Cálculo de tempos e indicadores
separacao["tempo_separacao_min"] = (separacao["ts_fim_separacao"] - separacao["ts_inicio_separacao"]).dt.total_seconds() / 60
kpi_sep = separacao.groupby("pedido_id").agg({
    "tempo_separacao_min": "mean",
    "status_separacao": lambda x: (x == "Concluído").mean() * 100
}).rename(columns={"tempo_separacao_min": "tempo_medio_separacao (min)", "status_separacao": "acuracidade_separacao (%)"})

expedicao["tempo_espera_expedicao_min"] = (expedicao["ts_inicio_carregamento"] - expedicao["ts_pronto_expedicao"]).dt.total_seconds() / 60
expedicao["tempo_total_expedicao_min"] = (expedicao["ts_fim_expedicao"] - expedicao["ts_pronto_expedicao"]).dt.total_seconds() / 60
expedicao["on_time_shipping (%)"] = expedicao["tempo_total_expedicao_min"] <= 240
expedicao["tempo_entre_sep_expedicao (min)"] = (expedicao["ts_inicio_carregamento"] - expedicao["ts_pronto_expedicao"]).dt.total_seconds() / 60

kpi_exp = expedicao.groupby("pedido_id").agg({
    "tempo_espera_expedicao_min": "mean",
    "tempo_entre_sep_expedicao (min)": "mean",
    "on_time_shipping (%)": "mean"
})

# Integração dos dados
indicadores = pedidos.merge(kpi_sep, on="pedido_id", how="inner").merge(kpi_exp, on="pedido_id", how="inner")
indicadores["data"] = indicadores["data_hora_criacao"].dt.date
indicadores["mes"] = indicadores["data_hora_criacao"].dt.to_period("M")

# KPIs mensais e projeção
media_mensal = indicadores.groupby("mes").agg({
    "tempo_medio_separacao (min)": "mean",
    "acuracidade_separacao (%)": "mean",
    "on_time_shipping (%)": "mean",
    "tempo_entre_sep_expedicao (min)": "mean"
}).reset_index()

projecoes = []
meses = pd.date_range(start="2025-04-01", end="2025-12-01", freq='MS').to_period("M")
ultimo = media_mensal.iloc[-1]
for mes in meses:
    mult_sep = 1.05
    mult_acc = 0.98
    if mes.month == 5:
        mult_sep = 1.15
        mult_acc = 0.90
    elif mes.month in [11, 12]:
        mult_sep = 1.25
        mult_acc = 0.85
    proj = {
        "mes": mes,
        "tempo_medio_separacao (min)": ultimo["tempo_medio_separacao (min)"] * mult_sep,
        "acuracidade_separacao (%)": ultimo["acuracidade_separacao (%)"] * mult_acc,
        "on_time_shipping (%)": ultimo["on_time_shipping (%)"] * 0.95,
        "tempo_entre_sep_expedicao (min)": ultimo["tempo_entre_sep_expedicao (min)"] * mult_sep
    }
    projecoes.append(proj)

proj_df = pd.concat([media_mensal, pd.DataFrame(projecoes)])
proj_df["mes"] = proj_df["mes"].astype(str)

# Gráficos principais
fig1 = px.line(proj_df, x="mes", y="tempo_medio_separacao (min)", title="Tempo Médio de Separação (min)")
fig1.update_yaxes(rangemode="tozero")
fig1.show()

fig2 = px.line(proj_df, x="mes", y="acuracidade_separacao (%)", title="Acuracidade de Separação (%)")
fig2.update_yaxes(rangemode="tozero")
fig2.show()

fig3 = px.line(proj_df, x="mes", y="on_time_shipping (%)", title="Expedição no Prazo (%)")
fig3.update_yaxes(rangemode="tozero")
fig3.show()

fig6 = px.line(proj_df, x="mes", y="tempo_entre_sep_expedicao (min)", title="Tempo entre Separação e Expedição (min)")
fig6.update_yaxes(rangemode="tozero")
fig6.show()

# Correlação entre quantidade de pedidos e erros
pedidos_por_dia = indicadores.groupby("data").size().reset_index(name="qtd_pedidos")
erros = separacao[separacao["status_separacao"] == "Erro"].copy()
erros["data"] = erros["ts_inicio_separacao"].dt.date
scatter = pd.merge(pedidos_por_dia, erros.groupby("data").size().reset_index(name="qtd_erros"), on="data", how="left")
scatter["qtd_erros"].fillna(0, inplace=True)
fig4 = px.scatter(scatter, x="qtd_pedidos", y="qtd_erros", title="Pedidos x Erros de Separação")
fig4.update_yaxes(rangemode="tozero")
fig4.update_traces(marker=dict(size=12))
fig4.show()

# Produtos com mais erros
erros_produtos = erros.groupby("produto_id").size().reset_index(name="qtd_erros").sort_values(by="qtd_erros", ascending=False).head(10)
fig5 = px.bar(erros_produtos, x="produto_id", y="qtd_erros", title="Top Produtos com Erros de Separação")
fig5.update_yaxes(rangemode="tozero")
fig5.show()

# Relatório com insights
print("\nINSIGHTS E AÇÕES PRIORITÁRIAS")
if proj_df["tempo_medio_separacao (min)"].iloc[-1] > 90:
    print("- O tempo médio de separação projetado para o final do ano ultrapassa 90 minutos. Reavaliar processos de picking e balanceamento da equipe.")
if proj_df["acuracidade_separacao (%)"].iloc[-1] < 95:
    print("- A acuracidade está projetada abaixo de 95%. Avaliar treinamentos e revisar causas de erro por produto e turno.")
if proj_df["on_time_shipping (%)"].mean() < 0.9:
    print("- A performance de expedição está abaixo de 90%. Considerar melhorias no planejamento de carregamento e no lead time de expedição.")
if scatter["qtd_erros"].max() > 10:
    print("- Há dias com mais de 10 erros. Investigar picos de volume e ajustar escala de trabalho ou suporte operacional.")


<ipython-input-71-223be9bd1cc1>:102: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.






INSIGHTS E AÇÕES PRIORITÁRIAS
- A acuracidade está projetada abaixo de 95%. Avaliar treinamentos e revisar causas de erro por produto e turno.
- Há dias com mais de 10 erros. Investigar picos de volume e ajustar escala de trabalho ou suporte operacional.


In [ ]:
# prompt: quantos pedidos temos por dia

# Contagem de pedidos por dia
pedidos_por_dia = indicadores.groupby("data").size().reset_index(name="quantidade_pedidos")

# Exibir os pedidos por dia
print("Quantidade de pedidos por dia:")
print(pedidos_por_dia)

# Opcional: Visualização da quantidade de pedidos por dia
fig_pedidos_dia = px.line(pedidos_por_dia, x="data", y="quantidade_pedidos", title="Quantidade de Pedidos por Dia")
fig_pedidos_dia.update_yaxes(rangemode="tozero")
fig_pedidos_dia.show()


Quantidade de pedidos por dia:
          data  quantidade_pedidos
0   2025-01-01                  57
1   2025-01-02                  66
2   2025-01-03                  53
3   2025-01-04                  63
4   2025-01-05                  54
..         ...                 ...
79  2025-03-21                  71
80  2025-03-22                  47
81  2025-03-23                  57
82  2025-03-24                  60
83  2025-03-25                  51

[84 rows x 2 columns]


In [ ]:

# Operador com maior quantidade de erros
erros_por_operador = erros.groupby("operador_id").size().reset_index(name="qtd_erros").sort_values(by="qtd_erros", ascending=False)

# Estatísticas da quantidade de erros por operador
media_erros = erros_por_operador["qtd_erros"].mean()
mediana_erros = erros_por_operador["qtd_erros"].median()
moda_erros = erros_por_operador["qtd_erros"].mode()

print("Estatísticas de Erros por Operador:")
print(f"Operador com mais erros:\n{erros_por_operador.head(1)}")
print(f"\nMédia de erros por operador: {media_erros:.2f}")
print(f"Mediana de erros por operador: {mediana_erros:.2f}")
print(f"Moda de erros por operador: {moda_erros.tolist()}")

fig7 = px.bar(erros_por_operador.head(10), x="operador_id", y="qtd_erros", title="Top 10 Operadores com Erros")
fig7.update_yaxes(rangemode="tozero")
fig7.show()


Estatísticas de Erros por Operador:
Operador com mais erros:
   operador_id  qtd_erros
15       OP016         28

Média de erros por operador: 21.45
Mediana de erros por operador: 21.50
Moda de erros por operador: [22]


In [ ]:
# prompt: VERIFIQUE SE HÁ ALGUMA CORRELAÇÃO DOS ERROS COM AS CIDADES DE DESTINO

# Verificar correlação entre erros e cidades de destino
# Unir os dados de erros com os dados de pedidos para obter a cidade de destino de cada erro
erros_com_cidade = erros.merge(pedidos[['pedido_id', 'destino']], on='pedido_id', how='left')

# Agrupar os erros por cidade de destino e contar a quantidade
erros_por_cidade = erros_com_cidade.groupby('destino').size().reset_index(name='qtd_erros_por_cidade')

# Opcional: Agrupar o total de pedidos por cidade para ter uma base de comparação
total_pedidos_por_cidade = pedidos.groupby('destino').size().reset_index(name='total_pedidos')

# Unir os dados de erros por cidade com o total de pedidos por cidade
erros_relativos_por_cidade = erros_por_cidade.merge(total_pedidos_por_cidade, on='destino', how='left')
erros_relativos_por_cidade['erros_porcentagem'] = (erros_relativos_por_cidade['qtd_erros_por_cidade'] / erros_relativos_por_cidade['total_pedidos']) * 100
erros_relativos_por_cidade.fillna(0, inplace=True) # Preenche NaN (cidades sem erros) com 0

# Exibir as cidades com maior quantidade absoluta de erros
print("\nCidades com maior quantidade absoluta de erros de separação:")
print(erros_por_cidade.sort_values(by='qtd_erros_por_cidade', ascending=False).head())

# Exibir as cidades com maior porcentagem de erros (pode ser mais relevante)
print("\nCidades com maior porcentagem de erros de separação (em relação ao total de pedidos para a cidade):")
# Filtra cidades que tiveram pelo menos um erro para evitar divisão por zero em cidades sem erros
erros_relativos_com_erros = erros_relativos_por_cidade[erros_relativos_por_cidade['qtd_erros_por_cidade'] > 0]
print(erros_relativos_com_erros.sort_values(by='erros_porcentagem', ascending=False).head())

# Visualização (Opcional): Gráfico de barras dos erros por cidade (Top N)
fig8 = px.bar(erros_por_cidade.sort_values(by='qtd_erros_por_cidade', ascending=False).head(10),
              x='destino', y='qtd_erros_por_cidade',
              title='Top 10 Cidades com Erros de Separação')
fig8.update_yaxes(rangemode="tozero")
fig8.show()

# Visualização (Opcional): Gráfico de barras da porcentagem de erros por cidade (Top N das que tiveram erro)
if not erros_relativos_com_erros.empty:
    fig9 = px.bar(erros_relativos_com_erros.sort_values(by='erros_porcentagem', ascending=False).head(10),
                  x='destino', y='erros_porcentagem',
                  title='Top 10 Cidades com Maior Porcentagem de Erros de Separação')
    fig9.update_yaxes(rangemode="tozero")
    fig9.show()
else:
    print("\nNão há cidades com erros para calcular a porcentagem.")

# Análise de Correlação: Embora um gráfico de barras nos mostre as cidades com mais erros,
# para verificar uma "correlação" formal, precisaríamos de dados de pedidos suficientes por cidade
# para garantir que a alta quantidade de erros não seja simplesmente devido a um alto volume de pedidos.
# A análise de porcentagem já dá uma ideia melhor disso.
# Podemos também calcular a correlação entre a quantidade TOTAL de pedidos por cidade e a quantidade TOTAL de erros por cidade.

# Mescla os dados para ter qtd_pedidos e qtd_erros por cidade
correlation_data = total_pedidos_por_cidade.merge(erros_por_cidade, on='destino', how='left')
correlation_data['qtd_erros_por_cidade'].fillna(0, inplace=True)

# Calcula a correlação de Pearson (mede a relação linear)
correlation_coefficient = correlation_data['total_pedidos'].corr(correlation_data['qtd_erros_por_cidade'])

print(f"\nCoeficiente de Correlação (Pearson) entre Total de Pedidos por Cidade e Quantidade de Erros por Cidade: {correlation_coefficient:.2f}")

# Interpretação do coeficiente de correlação:
# Um valor próximo a 1 indica uma forte correlação positiva (mais pedidos -> mais erros)
# Um valor próximo a -1 indica uma forte correlação negativa (mais pedidos -> menos erros)
# Um valor próximo a 0 indica pouca ou nenhuma correlação linear

if abs(correlation_coefficient) > 0.5:
    print("Sugere uma correlação moderada a forte entre o volume de pedidos e a quantidade de erros por cidade.")
elif abs(correlation_coefficient) > 0.2:
     print("Sugere uma correlação fraca a moderada entre o volume de pedidos e a quantidade de erros por cidade.")
else:
    print("Sugere uma correlação fraca ou inexistente entre o volume de pedidos e a quantidade de erros por cidade.")

# Visualização (Opcional): Scatter plot para visualizar a correlação
fig10 = px.scatter(correlation_data, x='total_pedidos', y='qtd_erros_por_cidade',
                   title='Relação entre Total de Pedidos e Erros por Cidade')
fig10.update_xaxes(title='Total de Pedidos por Cidade')
fig10.update_yaxes(title='Quantidade de Erros por Cidade', rangemode="tozero")
fig10.show()

print("\nAnálise da Correlação com Cidades de Destino:")
print("- O gráfico e o coeficiente de correlação indicam se há uma relação entre o volume de pedidos em uma cidade e a quantidade de erros.")
print("- Uma alta correlação positiva sugere que cidades com mais pedidos tendem a ter mais erros, o que pode indicar problemas de capacidade ou processo sob alto volume.")
print("- É importante analisar as cidades com maior porcentagem de erros para identificar problemas localizados que não estão diretamente ligados ao volume.")


Cidades com maior quantidade absoluta de erros de separação:
       destino  qtd_erros_por_cidade
0    Boa Vista                    96
3  Porto Velho                    92
1       Macapá                    87
2       Manaus                    78
4   Rio Branco                    76

Cidades com maior porcentagem de erros de separação (em relação ao total de pedidos para a cidade):
       destino  qtd_erros_por_cidade  total_pedidos  erros_porcentagem
0    Boa Vista                    96           1027           9.347614
1       Macapá                    87            964           9.024896
3  Porto Velho                    92           1023           8.993157
2       Manaus                    78            966           8.074534
4   Rio Branco                    76           1020           7.450980



Coeficiente de Correlação (Pearson) entre Total de Pedidos por Cidade e Quantidade de Erros por Cidade: 0.40
Sugere uma correlação fraca a moderada entre o volume de pedidos e a quantidade de erros por cidade.


<ipython-input-75-9356db771397>:53: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.






Análise da Correlação com Cidades de Destino:
- O gráfico e o coeficiente de correlação indicam se há uma relação entre o volume de pedidos em uma cidade e a quantidade de erros.
- Uma alta correlação positiva sugere que cidades com mais pedidos tendem a ter mais erros, o que pode indicar problemas de capacidade ou processo sob alto volume.
- É importante analisar as cidades com maior porcentagem de erros para identificar problemas localizados que não estão diretamente ligados ao volume.


In [ ]:
# prompt: calcule o tempo médio de espera para expedição

# Calcula o tempo médio de espera para expedição
tempo_medio_espera_expedicao = indicadores['tempo_espera_expedicao_min'].mean()

print(f"\nTempo Médio de Espera para Expedição: {tempo_medio_espera_expedicao:.2f} minutos")

# Opcional: Visualizar a distribuição do tempo de espera para expedição
fig_espera_exp = px.histogram(indicadores, x='tempo_espera_expedicao_min', nbins=50, title='Distribuição do Tempo de Espera para Expedição (min)')
fig_espera_exp.update_layout(xaxis_title='Tempo de Espera (min)', yaxis_title='Número de Pedidos')
fig_espera_exp.show()


Tempo Médio de Espera para Expedição: 97.59 minutos


In [ ]:
# prompt: faça uma correlação com string zona_armazem` no log separacao

# Correlação entre erros e zona_armazem
# Unir os dados de erros com os dados de log_separacao para obter a zona_armazem de cada erro
# Since 'zona_armazem' exists in both 'erros' (derived from separacao) and 'separacao',
# the merge will rename them to zona_armazem_x and zona_armazem_y.
# We want to use the zona_armazem from the separacao log associated with the error,
# which will be 'zona_armazem_y' after the merge.
erros_com_zona = erros.merge(separacao[['pedido_id', 'zona_armazem']], on='pedido_id', how='left', suffixes=('_x', '_y'))

# Agrupar os erros por zona_armazem (using the correct column from the merge) and count the quantity
erros_por_zona_armazem = erros_com_zona.groupby('zona_armazem_y').size().reset_index(name='qtd_erros_por_zona')

# Opcional: Agrupar o total de eventos de separação por zona_armazem para ter uma base de comparação
# Vamos usar o log_separacao original para contar quantas vezes cada zona_armazem aparece (como proxy para volume de trabalho)
total_eventos_por_zona = separacao.groupby('zona_armazem').size().reset_index(name='total_eventos_separacao')

# Unir os dados de erros por zona_armazem with the total events per zone
# Make sure to merge on the correct column name from the errors_por_zona_armazem dataframe
erros_relativos_por_zona = erros_por_zona_armazem.merge(total_eventos_por_zona, left_on='zona_armazem_y', right_on='zona_armazem', how='left')
erros_relativos_por_zona['erros_porcentagem_zona'] = (erros_relativos_por_zona['qtd_erros_por_zona'] / erros_relativos_por_zona['total_eventos_separacao']) * 100
erros_relativos_por_zona.fillna(0, inplace=True) # Preenche NaN (zonas sem erros) com 0

# Exibir as zonas com maior quantidade absoluta de erros
print("\nZonas de Armazém com maior quantidade absoluta de erros de separação:")
print(erros_por_zona_armazem.sort_values(by='qtd_erros_por_zona', ascending=False).head())

# Exibir as zonas com maior porcentagem de erros
print("\nZonas de Armazém com maior porcentagem de erros de separação (em relação ao total de eventos de separação na zona):")
# Filtra zonas que tiveram pelo menos um erro
erros_relativos_zona_com_erros = erros_relativos_por_zona[erros_relativos_por_zona['qtd_erros_por_zona'] > 0].copy() # Add .copy() to avoid SettingWithCopyWarning
print(erros_relativos_zona_com_erros.sort_values(by='erros_porcentagem_zona', ascending=False).head())

# Visualização (Opcional): Gráfico de barras dos erros por zona (Top N)
fig11 = px.bar(erros_por_zona_armazem.sort_values(by='qtd_erros_por_zona', ascending=False).head(10),
              x='zona_armazem_y', y='qtd_erros_por_zona',
              title='Top 10 Zonas de Armazém com Erros de Separação')
fig11.update_yaxes(rangemode="tozero")
fig11.show()

# Visualização (Opcional): Gráfico de barras da porcentagem de erros por zona (Top N das que tiveram erro)
if not erros_relativos_zona_com_erros.empty:
    fig12 = px.bar(erros_relativos_zona_com_erros.sort_values(by='erros_porcentagem_zona', ascending=False).head(10),
                  x='zona_armazem_y', y='erros_porcentagem_zona',
                  title='Top 10 Zonas de Armazém com Maior Porcentagem de Erros de Separação')
    fig12.update_yaxes(rangemode="tozero")
    fig12.show()
else:
    print("\nNão há zonas de armazém com erros para calcular a porcentagem.")

# Análise de Correlação formal entre o volume de eventos e a quantidade de erros por zona
# Ensure we use the correct column name from erros_por_zona_armazem for the merge
correlation_data_zona = total_eventos_por_zona.merge(erros_por_zona_armazem, left_on='zona_armazem', right_on='zona_armazem_y', how='left')
correlation_data_zona['qtd_erros_por_zona'].fillna(0, inplace=True)

# Calcula a correlação de Pearson
correlation_coefficient_zona = correlation_data_zona['total_eventos_separacao'].corr(correlation_data_zona['qtd_erros_por_zona'])

print(f"\nCoeficiente de Correlação (Pearson) entre Total de Eventos de Separação por Zona e Quantidade de Erros por Zona: {correlation_coefficient_zona:.2f}")

if abs(correlation_coefficient_zona) > 0.5:
    print("Sugere uma correlação moderada a forte entre o volume de eventos de separação e a quantidade de erros por zona de armazém.")
elif abs(correlation_coefficient_zona) > 0.2:
     print("Sugere uma correlação fraca a moderada entre o volume de eventos de separação e a quantidade de erros por zona de armazém.")
else:
    print("Sugere uma correlação fraca ou inexistente entre o volume de eventos de separação e a quantidade de erros por zona de armazém.")

# Visualização (Opcional): Scatter plot para visualizar a correlação
fig13 = px.scatter(correlation_data_zona, x='total_eventos_separacao', y='qtd_erros_por_zona',
                   title='Relação entre Total de Eventos de Separação e Erros por Zona de Armazém')
fig13.update_xaxes(title='Total de Eventos de Separação por Zona')
fig13.update_yaxes(title='Quantidade de Erros por Zona', rangemode="tozero")
fig13.show()

print("\nAnálise da Correlação com Zona de Armazém:")
print("- O gráfico e o coeficiente de correlação indicam se há uma relação entre a atividade em uma zona do armazém e a quantidade de erros.")
print("- Uma alta correlação positiva sugere que zonas com mais atividade tendem a ter mais erros, o que pode indicar problemas de layout, organização ou treinamento na zona.")
print("- É importante analisar as zonas com maior porcentagem de erros para identificar problemas localizados que não estão diretamente ligados ao volume de atividade geral da zona.")


Zonas de Armazém com maior quantidade absoluta de erros de separação:
  zona_armazem_y  qtd_erros_por_zona
1             A2                 339
2             B1                 331
3             B2                 321
0             A1                 309
4             C1                 282

Zonas de Armazém com maior porcentagem de erros de separação (em relação ao total de eventos de separação na zona):
  zona_armazem_y  qtd_erros_por_zona zona_armazem  total_eventos_separacao  \
1             A2                 339           A2                     2992   
2             B1                 331           B1                     2951   
3             B2                 321           B2                     3055   
0             A1                 309           A1                     2946   
4             C1                 282           C1                     2987   

   erros_porcentagem_zona  
1               11.330214  
2               11.216537  
3               10.507365  
0        


Coeficiente de Correlação (Pearson) entre Total de Eventos de Separação por Zona e Quantidade de Erros por Zona: 0.05
Sugere uma correlação fraca ou inexistente entre o volume de eventos de separação e a quantidade de erros por zona de armazém.


<ipython-input-82-6cdf68a89138>:54: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.






Análise da Correlação com Zona de Armazém:
- O gráfico e o coeficiente de correlação indicam se há uma relação entre a atividade em uma zona do armazém e a quantidade de erros.
- Uma alta correlação positiva sugere que zonas com mais atividade tendem a ter mais erros, o que pode indicar problemas de layout, organização ou treinamento na zona.
- É importante analisar as zonas com maior porcentagem de erros para identificar problemas localizados que não estão diretamente ligados ao volume de atividade geral da zona.


In [ ]:
# prompt: faça uma correlação da quantidade de pedidos por transportadora

# Agrupar os dados de expedição pela transportadora_id e contar a quantidade de pedidos associados a cada transportadora
# We use the 'expedicao' DataFrame as it contains the 'transportadora_id'
pedidos_por_transportadora = expedicao.groupby("transportadora_id").size().reset_index(name="quantidade_pedidos")

# Exibir a quantidade de pedidos por transportadora
print("\nQuantidade de pedidos por transportadora:")
print(pedidos_por_transportadora.sort_values(by="quantidade_pedidos", ascending=False))

# Opcional: Visualização da quantidade de pedidos por transportadora
fig_pedidos_transportadora = px.bar(pedidos_por_transportadora.sort_values(by="quantidade_pedidos", ascending=False),
                                     x="transportadora_id", y="quantidade_pedidos",
                                     title="Quantidade de Pedidos por Transportadora")
fig_pedidos_transportadora.update_yaxes(rangemode="tozero")
fig_pedidos_transportadora.show()

# Note: A correlação estatística (como Pearson) entre 'quantidade_pedidos' e 'transportadora_id' (que é categórico)
# não é diretamente aplicável ou significativa. A análise aqui é a contagem e visualização da distribuição de pedidos por transportadora.
# Se houver outros KPIs relacionados à transportadora (como tempo de entrega, ocorrências, etc.),
# seria possível fazer outras análises de correlação ou comparação de performance.


Quantidade de pedidos por transportadora:
  transportadora_id  quantidade_pedidos
1           TRANS02                1022
3           TRANS04                1022
4           TRANS05                1001
0           TRANS01                 999
2           TRANS03                 956


In [ ]:
# prompt: Tempo médio entre a criação do pedido e o fim da expedição:

# Merge pedidos with expedicao to get both creation time and end of expedition time
pedido_expedicao_times = pedidos[['pedido_id', 'data_hora_criacao']].merge(
    expedicao[['pedido_id', 'ts_fim_expedicao']], on='pedido_id', how='inner'
)

# Calculate the time difference
pedido_expedicao_times['tempo_criacao_fim_expedicao_min'] = (
    pedido_expedicao_times['ts_fim_expedicao'] - pedido_expedicao_times['data_hora_criacao']
).dt.total_seconds() / 60

# Calculate the average time
tempo_medio_criacao_fim_expedicao = pedido_expedicao_times['tempo_criacao_fim_expedicao_min'].mean()

print(f"\nTempo Médio entre a Criação do Pedido e o Fim da Expedição: {tempo_medio_criacao_fim_expedicao:.2f} minutos")

# Opcional: Visualizar a distribuição do tempo entre criação e fim da expedição
fig_criacao_exp = px.histogram(
    pedido_expedicao_times,
    x='tempo_criacao_fim_expedicao_min',
    nbins=50,
    title='Distribuição do Tempo entre Criação e Fim da Expedição (min)'
)
fig_criacao_exp.update_layout(xaxis_title='Tempo (min)', yaxis_title='Número de Pedidos')
fig_criacao_exp.show()


Tempo Médio entre a Criação do Pedido e o Fim da Expedição: 275.95 minutos


In [ ]:
# prompt: calcular o prazo de entrega para cada produto

# Calculate the estimated delivery time for each item
# We need to estimate the time from the end of expedition to delivery.
# This information is not directly available in the provided dataframes (pedidos, itens, separacao, expedicao).
# A realistic estimate would require historical data on delivery times per destination or transportadora.
# However, based on the available data, we can calculate the total time from order creation to the end of expedition.
# This gives us the internal processing time before the handoff to the transportadora.

# Merge itens with the aggregated pedido_expedicao_times to associate items with the order's internal processing time
# Include 'item_id' in the merge
itens_com_prazo_interno = itens[['item_pedido_id', 'pedido_id', 'produto_id', 'quantidade']].merge(
    pedido_expedicao_times[['pedido_id', 'tempo_criacao_fim_expedicao_min']],
    on='pedido_id',
    how='inner'
)

# Now, we need to estimate the external delivery time (from end of expedition to customer).
# Since we don't have this data, we will have to make an assumption or use external information.
# Let's assume an average external delivery time based on transportadora or destination, if possible.
# For this example, let's use the 'destino' from the 'pedidos' table to make a simplified estimate.
# We need to merge 'itens_com_prazo_interno' with 'pedidos' to get the 'destino' for each item.

itens_com_prazo_e_destino = itens_com_prazo_interno.merge(
    pedidos[['pedido_id', 'destino']],
    on='pedido_id',
    how='left' # Use left merge in case some items/orders have no destination in 'pedidos' (though unlikely with current data)
)

# Create a simplified mapping for estimated external delivery time based on destination.
# These are just example values. In a real scenario, this would come from historical delivery data.
# Let's assume delivery is slower to more distant/less populated destinations.
external_delivery_estimate_min = {
    'Manaus': 120, # 2 hours
    'Boa Vista': 480, # 8 hours
    'Porto Velho': 720, # 12 hours
    'Rio Branco': 960, # 16 hours
    'Macapá': 600 # 10 hours (adding Macapa based on the data)
}

# Apply the estimated external delivery time based on destination
itens_com_prazo_e_destino['tempo_entrega_externo_estimado_min'] = itens_com_prazo_e_destino['destino'].map(external_delivery_estimate_min).fillna(1440) # Defaulting to 24 hours if destination not in map

# Calculate the total estimated delivery time for each item
# Total estimated delivery time = Internal Processing Time + Estimated External Delivery Time
itens_com_prazo_e_destino['prazo_entrega_estimado_min'] = itens_com_prazo_e_destino['tempo_criacao_fim_expedicao_min'] + itens_com_prazo_e_destino['tempo_entrega_externo_estimado_min']

# Display the items with their estimated delivery time
# Let's select some relevant columns
produtos_com_prazo = itens_com_prazo_e_destino[['item_pedido_id', 'pedido_id', 'produto_id', 'destino', 'tempo_criacao_fim_expedicao_min', 'tempo_entrega_externo_estimado_min', 'prazo_entrega_estimado_min']].copy()

print("\nPrazo de Entrega Estimado para cada Item (em minutos):")
display(produtos_com_prazo.head()) # Use display() for better formatting

# You might want to aggregate this by pedido_id or produto_id for a summarized view
prazo_medio_por_produto = produtos_com_prazo.groupby('produto_id')['prazo_entrega_estimado_min'].mean().reset_index(name='prazo_medio_entrega_estimado_min')
print("\nPrazo Médio de Entrega Estimado por Produto (em minutos):")
display(prazo_medio_por_produto.sort_values(by='prazo_medio_entrega_estimado_min').head()) # Display products with shortest estimated delivery times

prazo_medio_por_destino = produtos_com_prazo.groupby('destino')['prazo_entrega_estimado_min'].mean().reset_index(name='prazo_medio_entrega_estimado_min')
print("\nPrazo Médio de Entrega Estimado por Destino (em minutos):")
display(prazo_medio_por_destino.sort_values(by='prazo_medio_entrega_estimado_min').head()) # Display destinations with shortest estimated delivery times

# Visualização (Opcional): Distribuição do prazo de entrega estimado
fig_prazo_entrega = px.histogram(produtos_com_prazo, x='prazo_entrega_estimado_min', nbins=50, title='Distribuição do Prazo de Entrega Estimado (min)')
fig_prazo_entrega.update_layout(xaxis_title='Prazo de Entrega Estimado (min)', yaxis_title='Número de Itens')
fig_prazo_entrega.show()

# Visualização (Opcional): Prazo médio de entrega estimado por destino
fig_prazo_destino = px.bar(prazo_medio_por_destino.sort_values(by='prazo_medio_entrega_estimado_min'),
                          x='destino', y='prazo_medio_entrega_estimado_min',
                          title='Prazo Médio de Entrega Estimado por Destino (min)')
fig_prazo_destino.update_yaxes(rangemode="tozero")
fig_prazo_destino.show()

print("\nNotas sobre o Prazo de Entrega Estimado:")
print("- O cálculo do prazo de entrega aqui é uma ESTIMATIVA, pois o tempo de trânsito EXTERNO (após a expedição) não está disponível nos dados fornecidos.")
print("- A estimativa externa foi baseada em um mapeamento manual simplificado por destino.")
print("- Para um cálculo de prazo de entrega preciso, seria necessário integrar dados de rastreamento ou informações das transportadoras sobre o tempo de trânsito real.")
print("- O 'prazo_entrega_estimado_min' representa o tempo total desde a criação do pedido até a entrega estimada ao cliente.")


Prazo de Entrega Estimado para cada Item (em minutos):


,item_pedido_id,pedido_id,produto_id,destino,tempo_criacao_fim_expedicao_min,tempo_entrega_externo_estimado_min,prazo_entrega_estimado_min
0,ITEM000001,PED00001,PROD0023,Manaus,323.0,120,443.0
1,ITEM000002,PED00001,PROD0008,Manaus,323.0,120,443.0
2,ITEM000003,PED00001,PROD0032,Manaus,323.0,120,443.0
3,ITEM000004,PED00001,PROD0067,Manaus,323.0,120,443.0
4,ITEM000005,PED00002,PROD0034,Boa Vista,365.0,480,845.0



Prazo Médio de Entrega Estimado por Produto (em minutos):


,produto_id,prazo_medio_entrega_estimado_min
26,PROD0027,811.973684
61,PROD0062,817.846154
48,PROD0049,818.487342
23,PROD0024,823.911950
13,PROD0014,824.225000



Prazo Médio de Entrega Estimado por Destino (em minutos):


,destino,prazo_medio_entrega_estimado_min
2,Manaus,402.764378
0,Boa Vista,765.369378
1,Macapá,883.340302
3,Porto Velho,1002.318227
4,Rio Branco,1242.627992



Notas sobre o Prazo de Entrega Estimado:
- O cálculo do prazo de entrega aqui é uma ESTIMATIVA, pois o tempo de trânsito EXTERNO (após a expedição) não está disponível nos dados fornecidos.
- A estimativa externa foi baseada em um mapeamento manual simplificado por destino.
- Para um cálculo de prazo de entrega preciso, seria necessário integrar dados de rastreamento ou informações das transportadoras sobre o tempo de trânsito real.
- O 'prazo_entrega_estimado_min' representa o tempo total desde a criação do pedido até a entrega estimada ao cliente.


# STEP 2 - CLARIFICANDO O PROBLEMA

Investigando o problema, identificando a causa raiz e entendendo os fatores que contribuem para ele.

INSIGHTS



- A acuracidade de separação está projetada abaixo de 95%. Avaliar treinamentos e revisar causas de erro por produto e turno.
- Há dias com mais de 10 erros. Investigar picos de volume e ajustar escala de trabalho ou suporte operacional.
- Há 3 operadores com erros acima da moda (22)
- Os dias com mais erros não foram os dias com mais pedidos porém foram dias com mais de 60 pedidos.
- Há 3 produtos com maior incidência de erros;
- Boa Vista é a cidade com maior erros de separação;
- Após a separação há um tempo de 97 minutos até a expedição, suponho que seja o processo de embalagem
- O armazém A2 é o armazém onde possui mais erros
- Tempo Médio entre a Criação do Pedido e o Fim da Expedição: 275.95 minutos
- Rio Branco é a cidade com o maior lead time de entrega

# Avaliação de padrões, anomalias e tendências com base no dashboard

## 1. **Padrões Observados**
- **Curva previsível de aumento do tempo de separação em meses sazonais**: maio, novembro e dezembro estão projetados com aumentos consistentes nos tempos médios, o que segue o comportamento esperado de sazonalidade.
- **Consistência nas metas de "on-time shipping" até março**: mantém valores acima de 90% nos meses observados, com queda projetada para meses de pico.

## 2. **Anomalias Identificadas**
- **Acuracidade de separação abaixo de 95%** ao longo da projeção anual, com queda ainda maior em maio (90%) e nos meses de fim de ano (85%). Isso indica falhas sistemáticas em meses de alta demanda.
- **Disparidade entre número de pedidos e erros**: não há correlação direta entre volume de pedidos e número de erros, o que pode indicar problemas de gestão da equipe ou falhas em produtos específicos.
- **Erros concentrados em três produtos específicos e em três operadores**: indicativo de necessidade de treinamento direcionado ou revisão de layout de picking.

## 3. **Tendências**
- **Crescimento constante no tempo entre separação e expedição**: embora esperado em sazonalidade, deve-se analisar a capacidade da área de embalagem, pois esse tempo médio está em 97 minutos.
- **Erros concentrados no armazém A2**: indica gargalo logístico ou falhas de processo específicas neste local.
- **Cidade de Boa Vista com maior ocorrência de erros**: pode sinalizar problema em layout logístico de destino ou complexidade de pedido específico para esta praça.
- **Tempo total do pedido até a expedição é de aproximadamente 275 minutos**: bom para operações urbanas, mas há margem de ganho se processos forem otimizados.



# STEP 03 - IDENTIFICANDO AS POSSÍVEIS SOLUÇÕES

Brainstorming e exploração diversas soluções possíveis para resolver o problema identificado
---

### 🎯 **KGI – Indicadores Estratégicos**

#### 1. **Eficiência Logística Global (Tempo Total do Pedido: 275,95 min)**

* 📌 **Ação 1**: Implementar um sistema de priorização automática de pedidos com base na janela de entrega e SLA esperado.
* 📌 **Ação 2**: Reavaliar o fluxo de processos entre separação e expedição, com foco em sincronização entre áreas.

#### 2. **Confiabilidade no Atendimento (SLA abaixo de 95%)**

* 📌 **Ação 1**: Revisar e otimizar o cronograma de carregamento, com buffers para absorver atrasos operacionais.
* 📌 **Ação 2**: Criar alertas automáticos de SLA próximo ao vencimento, acionando operadores-chave.

#### 3. **Redução de Custos por Erro Operacional**

* 📌 **Ação 1**: Aplicar gamificação e feedback semanal individualizado com ranking por acuracidade.
* 📌 **Ação 2**: Automatizar o check de separação com sistemas de conferência por leitura RFID ou câmera.

---

### 📈 **KPI – Indicadores de Performance**

#### 1. **Tempo Médio de Separação (\~90 min)**

* 📌 **Ação 1**: Revisar layout de armazenagem para otimizar rotas de picking (análise ABC dos produtos).
* 📌 **Ação 2**: Implementar sistema de picking por voz ou assistente visual para aumentar velocidade com precisão.

#### 2. **Acuracidade de Separação (<95%)**

* 📌 **Ação 1**: Realizar requalificação dos operadores com alto índice de erro em produtos críticos.
* 📌 **Ação 2**: Introduzir duplo check nos itens com maior taxa de erro antes do envio à embalagem.

#### 3. **SLA de Pedido no Prazo (<90% projetado)**

* 📌 **Ação 1**: Integrar o sistema de WMS com TMS para prever gargalos e antecipar falhas de expedição.
* 📌 **Ação 2**: Alocar contingência de recursos (reserva de slots de carga) em períodos sazonais.

#### 4. **Tempo entre Separação e Expedição (\~97 min)**

* 📌 **Ação 1**: Inserir monitoramento da fila de pedidos prontos, priorizando pedidos por tempo parado.
* 📌 **Ação 2**: Revisar recursos e layout da área de embalagem para evitar acúmulo e lentidão.

#### 5. **Pedidos por Dia (variação com picos >60)**

* 📌 **Ação 1**: Ajustar turnos e escalas de acordo com curva de demanda real histórica (previsão por data comemorativa).
* 📌 **Ação 2**: Criar times móveis de reforço que atuem em dias de pico com atuação em áreas críticas.

---

### 🧩 **KAI – Indicadores de Atividade**

#### 1. **Erros por Operador (3 com erros acima da moda)**

* 📌 **Ação 1**: Estabelecer metas de qualidade individuais com reconhecimento para evolução.
* 📌 **Ação 2**: Aplicar rodízio de tarefas para entender se o erro está associado à função ou operador.

#### 2. **Erros por Produto (top 3 com reincidência)**

* 📌 **Ação 1**: Sinalizar visualmente os produtos de difícil separação com etiquetas diferenciadas.
* 📌 **Ação 2**: Avaliar se a descrição ou codificação do produto está confusa ou ambígua.

#### 3. **Erros por Turno e Dia (não correlacionados a maior volume)**

* 📌 **Ação 1**: Reforçar presença de supervisão nos turnos com maior erro, independentemente do volume.
* 📌 **Ação 2**: Criar rotina de revisão dos primeiros 30 minutos de operação (aferição de alinhamento e metas).

#### 4. **Erros por Armazém (A2 liderando)**

* 📌 **Ação 1**: Realizar auditoria no layout e organização do armazém A2.
* 📌 **Ação 2**: Reavaliar a capacitação da equipe alocada exclusivamente nesse armazém.

#### 5. **Tempo de Espera na Expedição (\~97 min)**

* 📌 **Ação 1**: Mapear microprocessos entre separação e expedição (ex.: embalagem e conferência).
* 📌 **Ação 2**: Avaliar a possibilidade de automação parcial com esteiras ou direcionamento automático.






# STEP 4 - AVALIAR ESFORÇO X IMPACTO

Analise os prós e os contras de cada solução, considerando fatores como viabilidade, custo e impacto em outras áreas. Escolha a solução mais eficaz com base na sua análise.


# STEP 5 - PRIORIZAR - SELECIONAR AS MELHORES SOLUÇÕES

Com base na avaliação, escolha a solução que melhor aborda o problema e atende aos seus critérios. Desenvolva um plano de ação para implementação.

# STEP 6 - IMPLEMENTAR AS SOLUÇÕES

Coloque a solução escolhida em prática. Isso envolve criar um plano detalhado, alocar recursos e garantir que a solução seja implementada de forma eficaz.


# STEP 7 - AVALIAÇÃO DE RESULTADOS

Monitore a implementação e avalie se a solução resolveu o problema com sucesso. Faça os ajustes necessários para garantir o sucesso contínuo e planeje ocorrências futuras.
